In [1]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
from crewai import Agent, Crew, Task, LLM
import os
from dotenv import load_dotenv
load_dotenv()

# Mistral LLM Setup
llm = LLM(
    model="mistral/mistral-small-latest",
    api_key=os.getenv("MISTRAL_API_KEY")
)

## Role Playing, Focus and Cooperation

In [4]:
support_agent = Agent(
    role = 'Senior Support Representative',
    goal = "Be the most helpful and knowledgeable support agent in the team.",
    backstory= (
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
    ),
    instructions=(
        "You may ONLY use the tools provided. "
        "Do NOT attempt to call tools that are not listed in your toolset. "
        "There is NO 'add_image' tool. Focus on using 'ScrapeWebsiteTool' only. "
        "If unsure, explain it with text — do NOT invent tools."
    ),
    allow_delegation=False,
    verbose = True,
    llm = llm
)

- By default the allow_delegation has value as True which allows the agents to delegate work to the agent which is best suitated for the task.

In [109]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True,
    allow_delegation= True,
    llm = llm
)

## Tools, Guardrails and Memory

### Tools

- Some ways of using CrewAI tools.

```Python
search_tool = SerperDevTool()  # Detect every link that it sees
scrape_tool = ScrapeWebsiteTool() # Detect only the particualr link

In [110]:
# in custom_tools.py
from crewai_tools.tools.scrape_website_tool.scrape_website_tool import ScrapeWebsiteTool
from crewai_tools.tools.serper_dev_tool.serper_dev_tool import SerperDevTool
from crewai_tools.tools.website_search.website_search_tool import WebsiteSearchTool


In [111]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

### Creating Tasks
- You are passing the Tool on the Task Level.

In [112]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "You **MUST** use only the ScrapeWebsiteTool provided. "
        "Do NOT invent or use any tool that is not explicitly provided. "
        "There is NO 'add_image' tool available. "
        "You are expected to respond ONLY using valid tools or text output."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

- `quality_assurance_review` is not using any Tool(s)
- Here the QA Agent will only review the work of the Support Agent

In [113]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [114]:
crew = Crew(
    agents = [support_agent, support_quality_assurance_agent],
    tasks = [inquiry_resolution, quality_assurance_review],
    verbose = True,
    memory = False
)

In [ ]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

In [ ]:
from IPython.display import Markdown
Markdown(result)